This notebook is to generate mel spectrograms for 2 second utterances by chunking the original signal in 2 second frames.

In [1]:
import os
import librosa
import numpy as np
import sys
import pickle

In [2]:
data_folder = './13_language_dataset/'
num_langs = len([name for name in os.listdir(data_folder) if not name.startswith('.')])

l=0
data={}
labels={}
for lang in os.listdir(data_folder):
    flag=False
    f=1
    if not lang.startswith('.'):
        for wavfile in os.listdir(data_folder+lang):
            num_files = len([name for name in os.listdir(data_folder+lang) if not name.startswith('.')])
            sig, fs = librosa.load(data_folder+lang+'/'+wavfile,sr=None)
            for i in range(round(sig.shape[0]/(2*fs))): #to divide the signal into 2 second signals
                y=sig[2*fs*i:2*fs*(i+1)-1] #taking 2 seconds only
                
                a = 0.97
                emp_sig = np.append(y[0],y[1:] - a*y[:-1]) #emphasized signal y(t)=x(t)-a*x(t-1)

                win_length = int(round(0.025*fs)) # for 25 ms frame length
                hop_length = int(round(0.01*fs)) # for 10ms shift between frames
                d = librosa.stft(emp_sig,n_fft=512,win_length=win_length,hop_length=hop_length)
                D = np.abs(d)**2 #to get power spectrum
                S = librosa.feature.melspectrogram(S=D)

                if flag is False:
                    data[l]=[]
                    labels[l]=[]
                    flag=True
            
                data[l].append(S)
                labels[l].append(l)
            
            if (f%51) is 1: #To print status after every 51 generated spectrograms
                print('Generating spectrograms for {} ({}/{} wavfiles done). {} languages remaining'.format(lang,f,num_files,num_langs-l-1))

            f=f+1
        l=l+1


Generating spectrograms for manipuri (1/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (52/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (103/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (154/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (205/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (256/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (307/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (358/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (409/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (460/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (511/628 wavfiles done). 12 languages remaining
Generating spectrograms for manipuri (562/628 wavfiles done). 12 lan

Generating spectrograms for telugu (460/628 wavfiles done). 5 languages remaining
Generating spectrograms for telugu (511/628 wavfiles done). 5 languages remaining
Generating spectrograms for telugu (562/628 wavfiles done). 5 languages remaining
Generating spectrograms for telugu (613/628 wavfiles done). 5 languages remaining
Generating spectrograms for marathi (1/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (52/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (103/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (154/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (205/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (256/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (307/628 wavfiles done). 4 languages remaining
Generating spectrograms for marathi (358/628 wavfiles done). 4 languages remaining
Generating 

In [3]:
class_names = []
for lang in os.listdir(data_folder):
    if not lang.startswith('.'):
        class_names.append(lang)

class_names = np.asarray(class_names)
print('Languages to be classified and available in the dataset : {}'.format(class_names))

Languages to be classified and available in the dataset : ['manipuri' 'urdu' 'bengali' 'kannada' 'odia' 'punjabi' 'tamil' 'telugu'
 'marathi' 'gujarati' 'malayalam' 'hindi' 'assamese']


In [ ]:
for l in range(class_names.shape[0]):
    if l is 0:
        corpus = np.asarray(data[l])
        targets = np.asarray(labels[l])
    else:
        corpus = np.vstack((corpus,data[l]))
        targets = np.hstack((targets,labels[l]))

print('Dataset dimensions :')
print('Number of files = {}'.format(corpus.shape[0]))
print('Dimensions of each file = {}'.format(corpus[0].shape))
print('Targets = {}'.format(targets.shape[0]))

corpus = np.reshape(corpus,(corpus.shape[0],corpus.shape[1],corpus.shape[2],1))
print(corpus.shape)

with open('13L_2sec_mel_spectrograms.pickle','wb') as f:
    pickle.dump((corpus,targets),f)

Dataset dimensions :
Number of files = 16328
Dimensions of each file = (128, 200)
Targets = 16328
(16328, 128, 200, 1)
